# Removing Controls from Samples

In [2]:
import pandas as pd

def load_and_clean(file_path):
    return pd.read_csv(file_path)

def remove_shared_variants(case_df, control_df):
    initial_count = len(case_df)
    # Create a set of tuples containing the control variants (Chr, Coordinate1, Coordinate2, Type)
    control_variants = set(control_df[['chr', 'Coordinate1', 'Coordinate2', 'Type']].itertuples(index=False, name=None))
    # Filter out rows in case_df that are in control_variants
    filtered_df = case_df[~case_df.apply(lambda row: (row['chr'], row['Coordinate1'], row['Coordinate2'], row['Type']) in control_variants, axis=1)]
    removed_count = initial_count - len(filtered_df)
    remaining_count = len(filtered_df)
    return filtered_df, removed_count, remaining_count

# Load the data
controls = load_and_clean('Controls/Controls_V1.csv')
hi_cases = load_and_clean('HI_V1.csv')
ndm_cases = load_and_clean('NDM_V1.csv')

# Remove shared variants
filtered_hi_cases, hi_removed, hi_remaining = remove_shared_variants(hi_cases, controls)
filtered_ndm_cases, ndm_removed, ndm_remaining = remove_shared_variants(ndm_cases, controls)

# Save the filtered data after removing shared variants
filtered_hi_cases.to_csv('HI_V2.csv', index=False)
filtered_ndm_cases.to_csv('NDM_V2.csv', index=False)

# Output the counts
hi_counts = {
    "Initial Count": len(hi_cases),
    "Removed Count (Shared Variants)": hi_removed,
    "Remaining Count (After Shared Variants)": hi_remaining
}

ndm_counts = {
    "Initial Count": len(ndm_cases),
    "Removed Count (Shared Variants)": ndm_removed,
    "Remaining Count (After Shared Variants)": ndm_remaining
}

print("HI Case Counts:", hi_counts)
print("NDM Case Counts:", ndm_counts)


/var/folders/c9/5211zt715d7151h7fkq093h40000gn/T/ipykernel_82628/1459858627.py:4: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


HI Case Counts: {'Initial Count': 576293, 'Removed Count (Shared Variants)': 149328, 'Remaining Count (After Shared Variants)': 426965}
NDM Case Counts: {'Initial Count': 526289, 'Removed Count (Shared Variants)': 454, 'Remaining Count (After Shared Variants)': 525835}


# 25kb Variants - V3

In [14]:
import pandas as pd

def load_and_clean(file_path):
    return pd.read_csv(file_path)

def remove_large_variants(df):
    initial_count = len(df)
    # Filter variants with Read_Width within the range -25000 to 25000
    filtered_df = df[(df['Read_Width'] <= 25000) & (df['Read_Width'] >= -25000)]
    removed_count = initial_count - len(filtered_df)
    remaining_count = len(filtered_df)
    return filtered_df, removed_count, remaining_count

# Load the data
hi_cases = load_and_clean('HI_V2.csv')
ndm_cases = load_and_clean('NDM_V2.csv')

# Remove large variants
filtered_hi_cases, hi_large_removed, hi_large_remaining = remove_large_variants(hi_cases)
filtered_ndm_cases, ndm_large_removed, ndm_large_remaining = remove_large_variants(ndm_cases)

# Save the filtered data
filtered_hi_cases.to_csv('HI_V3.csv', index=False)
filtered_ndm_cases.to_csv('NDM_V3.csv', index=False)

# Output the counts
hi_counts = {
    "Initial Count": len(hi_cases),
    "Removed Count (Large Variants)": hi_large_removed,
    "Final Remaining Count": hi_large_remaining
}

ndm_counts = {
    "Initial Count": len(ndm_cases),
    "Removed Count (Large Variants)": ndm_large_removed,
    "Final Remaining Count": ndm_large_remaining
}

print("HI Case Counts:", hi_counts)
print("NDM Case Counts:", ndm_counts)



C:\Users\University\AppData\Local\Temp\ipykernel_29664\1468906388.py:4: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


HI Case Counts: {'Initial Count': 426965, 'Removed Count (Large Variants)': 178477, 'Final Remaining Count': 248488}
NDM Case Counts: {'Initial Count': 525835, 'Removed Count (Large Variants)': 172378, 'Final Remaining Count': 353457}


# Identifying DGV Overlaps via Bedtools on the command line

In [ ]:
sort -k1,1 -k2,2n dgvGold_unique.bed > dgvGold_sorted.bed

bedtools merge -i dgvGold_sorted.bed > dgvGold_merged.bed

bedtools intersect -a /mnt/data/NDM_V3_corrected_from_csv.bed -b dgvGold_merged.bed -f 0.9 > NDM_DGV_common_strict.bed

# Adjusting negative coordinates and adding chr prefix

In [2]:
import pandas as pd

# Load the CSV files
ndm_v3_path = 'NDM_V3.csv'
hi_v3_path = 'HI_V3.csv'

ndm_v3_data = pd.read_csv(ndm_v3_path)
hi_v3_data = pd.read_csv(hi_v3_path)

# Function to correct coordinates and add 'chr' prefix
def correct_coordinates(data):
    # Swap start and end positions where necessary
    data.loc[data['Coordinate2'] < data['Coordinate1'], ['Coordinate1', 'Coordinate2']] = data.loc[data['Coordinate2'] < data['Coordinate1'], ['Coordinate2', 'Coordinate1']].values
    # Prefix chromosome numbers with 'chr'
    data['Chr'] = 'chr' + data['Chr'].astype(str)
    return data

# Correct both datasets
corrected_ndm_v3_data = correct_coordinates(ndm_v3_data)
corrected_hi_v3_data = correct_coordinates(hi_v3_data)

# Save the corrected data to new CSV files
corrected_ndm_v3_path = 'NDM_V3_corrected.csv'
corrected_hi_v3_path = 'HI_V3_corrected.csv'

corrected_ndm_v3_data.to_csv(corrected_ndm_v3_path, index=False)
corrected_hi_v3_data.to_csv(corrected_hi_v3_path, index=False)



# Identifying Unique Variants from DGV

In [3]:
import pandas as pd
import os

# Paths to your CSV files
ndm_csv_path = 'NDM_V3.csv'
hi_csv_path = 'HI_V3.csv'

# Load the CSV files
ndm_csv_data = pd.read_csv(ndm_csv_path)
hi_csv_data = pd.read_csv(hi_csv_path)

# Function to correct coordinates and add 'chr' prefix
def correct_coordinates(data):
    # Swap start and end positions where necessary
    data.loc[data['Coordinate2'] < data['Coordinate1'], ['Coordinate1', 'Coordinate2']] = data.loc[data['Coordinate2'] < data['Coordinate1'], ['Coordinate2', 'Coordinate1']].values
    # Prefix chromosome numbers with 'chr'
    data['Chr'] = 'chr' + data['Chr'].astype(str)
    return data

# Correct both datasets
corrected_ndm_v3_data = correct_coordinates(ndm_csv_data)
corrected_hi_v3_data = correct_coordinates(hi_csv_data)

# Function to convert to BED format with extra information
def convert_to_bed_with_extra_info(data, output_path):
    # Ensure correct order of columns: chr, start, end, and extra columns
    bed_data = data[['Chr', 'Coordinate1', 'Coordinate2'] + [col for col in data.columns if col not in ['Chr', 'Coordinate1', 'Coordinate2']]]
    bed_data.to_csv(output_path, sep='\t', header=False, index=False)

# Define output paths for BED files
ndm_bed_path = 'NDM_V4.bed'
hi_bed_path = 'HI_V4.bed'

# Convert to BED format
convert_to_bed_with_extra_info(corrected_ndm_v3_data, ndm_bed_path)
convert_to_bed_with_extra_info(corrected_hi_v3_data, hi_bed_path)

# Sort the BED files
os.system(f"sort -k1,1 -k2,2n {ndm_bed_path} > {ndm_bed_path.replace('.bed', '_sorted.bed')}")
os.system(f"sort -k1,1 -k2,2n {hi_bed_path} > {hi_bed_path.replace('.bed', '_sorted.bed')}")

# Define paths for the sorted BED files and the DGV data
ndm_bed_sorted_path = ndm_bed_path.replace('.bed', '_sorted.bed')
hi_bed_sorted_path = hi_bed_path.replace('.bed', '_sorted.bed')
dgv_bed_path = 'DGV/dgvGold.bed'

# Ensure DGV data is sorted
os.system(f"sort -k1,1 -k2,2n {dgv_bed_path} > {dgv_bed_path.replace('.bed', '_sorted.bed')}")
dgv_bed_sorted_path = dgv_bed_path.replace('.bed', '_sorted.bed')

# Use Bedtools Subtract to find unique intervals
os.system(f"bedtools subtract -a {ndm_bed_sorted_path} -b {dgv_bed_sorted_path} -f 0.9 > {ndm_bed_sorted_path.replace('_sorted.bed', '_unique.bed')}")
os.system(f"bedtools subtract -a {hi_bed_sorted_path} -b {dgv_bed_sorted_path} -f 0.9 > {hi_bed_sorted_path.replace('_sorted.bed', '_unique.bed')}")

# Paths for the unique BED files
ndm_bed_unique_path = ndm_bed_sorted_path.replace('_sorted.bed', '_unique.bed')
hi_bed_unique_path = hi_bed_sorted_path.replace('_sorted.bed', '_unique.bed')

# Function to convert BED back to CSV
def bed_to_csv(bed_path, original_csv, output_csv):
    bed_data = pd.read_csv(bed_path, sep='\t', header=None)
    bed_data.columns = original_csv.columns  # Use the original column names
    bed_data.to_csv(output_csv, index=False)

# Convert the unique BED files back to CSV
bed_to_csv(ndm_bed_unique_path, corrected_ndm_v3_data, ndm_bed_unique_path.replace('.bed', '.csv'))
bed_to_csv(hi_bed_unique_path, corrected_hi_v3_data, hi_bed_unique_path.replace('.bed', '.csv'))

print(f"Unique intervals for NDM saved to {ndm_bed_sorted_path.replace('_sorted.bed', '_unique.csv')}")
print(f"Unique intervals for HI saved to {hi_bed_sorted_path.replace('_sorted.bed', '_unique.csv')}")

Unique intervals for NDM saved to NDM_V4_unique.csv
Unique intervals for HI saved to HI_V4_unique.csv


# Remove variants shared by NDM & HI - V5

In [9]:
import pandas as pd

# Paths to the split CSV files
ndm_split_csv_path = 'NDM_V4_unique.csv'
hi_split_csv_path = 'HI_V4_unique.csv'

# Load the split CSV files into DataFrames
ndm_csv_data = pd.read_csv(ndm_split_csv_path)
hi_csv_data = pd.read_csv(hi_split_csv_path)

# Function to find unique variants by type
def find_unique_variants_by_type(primary_df, subtract_df, variant_type):
    primary_df_type = primary_df[primary_df['Type'] == variant_type]
    subtract_df_type = subtract_df[subtract_df['Type'] == variant_type]
    
    # Merge and find unique variants within this type
    merged_df = pd.merge(primary_df_type, subtract_df_type, on=['Chr', 'Coordinate1', 'Coordinate2'], how='left', indicator=True)
    
    # Filter to keep only unique rows from the primary DataFrame
    unique_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Drop the extra columns that were suffixed with '_y'
    unique_df = unique_df.loc[:, ~unique_df.columns.str.endswith('_y')]
    
    # Rename the '_x' columns back to their original names
    unique_df.columns = unique_df.columns.str.replace('_x', '', regex=False)
    
    return unique_df

# Variant types to consider (e.g., 'deletion', 'duplication')
variant_types = ndm_csv_data['Type'].unique()

# Initialize empty DataFrames to collect unique variants for each type
ndm_unique_variants_all = pd.DataFrame()
hi_unique_variants_all = pd.DataFrame()

# Loop over each variant type
for variant_type in variant_types:
    # Find NDM unique variants by subtracting HI variants of the same type
    ndm_unique_variants = find_unique_variants_by_type(ndm_csv_data, hi_csv_data, variant_type)
    
    # Find HI unique variants by subtracting NDM variants of the same type
    hi_unique_variants = find_unique_variants_by_type(hi_csv_data, ndm_csv_data, variant_type)
    
    # Append the results for this variant type
    ndm_unique_variants_all = pd.concat([ndm_unique_variants_all, ndm_unique_variants])
    hi_unique_variants_all = pd.concat([hi_unique_variants_all, hi_unique_variants])


# Save the results back to CSV format
ndm_unique_csv_path = 'NDM_V5_unique.csv'
hi_unique_csv_path = 'HI_V5_unique.csv'

ndm_unique_variants_all.to_csv(ndm_unique_csv_path, index=False)
hi_unique_variants_all.to_csv(hi_unique_csv_path, index=False)

print(f"NDM unique variants by type saved to {ndm_unique_csv_path}")
print(f"HI unique variants by type saved to {hi_unique_csv_path}")

NDM unique variants by type saved to NDM_V5_unique.csv
HI unique variants by type saved to HI_V5_unique.csv


# DeNovos

In [10]:
import pandas as pd

# Paths to the final unique variants CSV files
ndm_unique_csv_path = 'NDM_V5_unique.csv'
hi_unique_csv_path = 'HI_V5_unique.csv'

# Load the CSV files into DataFrames
ndm_csv_data = pd.read_csv(ndm_unique_csv_path)
hi_csv_data = pd.read_csv(hi_unique_csv_path)

# Function to filter for DeNovo variants (GT == '0/0')
def filter_de_novo_variants(df):
    de_novo_df = df[df['GT'] == '0/0']
    return de_novo_df

# Filter for DeNovo variants in both datasets
ndm_de_novo_variants = filter_de_novo_variants(ndm_csv_data)
hi_de_novo_variants = filter_de_novo_variants(hi_csv_data)

# Save the DeNovo variants to new CSV files
ndm_de_novo_csv_path = 'NDM_DeNovo.csv'
hi_de_novo_csv_path = 'HI_DeNovo.csv'

ndm_de_novo_variants.to_csv(ndm_de_novo_csv_path, index=False)
hi_de_novo_variants.to_csv(hi_de_novo_csv_path, index=False)

print(f"NDM DeNovo variants saved to {ndm_de_novo_csv_path}")
print(f"HI DeNovo variants saved to {hi_de_novo_csv_path}")

NDM DeNovo variants saved to NDM_DeNovo.csv
HI DeNovo variants saved to HI_DeNovo.csv


# Filter by Read Count

In [13]:
import pandas as pd

# Paths to the DeNovo variants CSV files
ndm_de_novo_csv_path = 'NDM_DeNovo.csv'
hi_de_novo_csv_path = 'HI_DeNovo.csv'

# Load the CSV files into DataFrames
ndm_csv_data = pd.read_csv(ndm_de_novo_csv_path)
hi_csv_data = pd.read_csv(hi_de_novo_csv_path)

# Function to filter variants by read count (>= 7)
def filter_by_read_count(df):
    filtered_df = df[df['Read_Count'] >= 5]
    return filtered_df

# Apply the filter to both datasets
ndm_filtered_variants = filter_by_read_count(ndm_csv_data)
hi_filtered_variants = filter_by_read_count(hi_csv_data)

# Save the filtered variants to new CSV files
ndm_filtered_csv_path = 'NDM_DeNovo5.csv'
hi_filtered_csv_path = 'HI_DeNovo5.csv'

ndm_filtered_variants.to_csv(ndm_filtered_csv_path, index=False)
hi_filtered_variants.to_csv(hi_filtered_csv_path, index=False)

print(f"NDM variants filtered by read count saved to {ndm_filtered_csv_path}")
print(f"HI variants filtered by read count saved to {hi_filtered_csv_path}")

NDM variants filtered by read count saved to NDM_DeNovo5.csv
HI variants filtered by read count saved to HI_DeNovo5.csv


# Shared Variants within disease groups

In [14]:
import pandas as pd

# Load the DeNovo CSV files
ndm_csv_data = pd.read_csv('NDM_DeNovo5.csv')
hi_csv_data = pd.read_csv('HI_DeNovo5.csv')

# Function to identify shared variants with overlaps
def identify_shared_variants(df):
    # Sort by Chr, Coordinate1, and Coordinate2
    df = df.sort_values(by=['Chr', 'Region1', 'Region2'])
    
    # Create an empty DataFrame to store results
    shared_variants = pd.DataFrame()
    
    # Loop through each chromosome
    for chr in df['Chr'].unique():
        chr_df = df[df['Chr'] == chr]
        
        # Initialize a list to hold overlapping groups
        overlapping_groups = []
        
        # Initialize the first group with the first row
        current_group = [chr_df.iloc[0]]
        
        for i in range(1, len(chr_df)):
            # Check if the current variant overlaps with the previous one
            prev_row = chr_df.iloc[i-1]
            curr_row = chr_df.iloc[i]
            
            if curr_row['Region1'] <= prev_row['Region2']:
                # If they overlap, add the current row to the group
                current_group.append(curr_row)
            else:
                # If they don't overlap, finalize the current group and start a new one
                overlapping_groups.append(pd.DataFrame(current_group))
                current_group = [curr_row]
        
        # Add the last group
        overlapping_groups.append(pd.DataFrame(current_group))
        
        # For each group of overlapping variants, count unique Sample_IDs
        for group in overlapping_groups:
            if group['Sample_ID'].nunique() > 1:
                shared_variants = pd.concat([shared_variants, group])
    
    return shared_variants

# Apply the function to both datasets
ndm_shared_variants = identify_shared_variants(ndm_csv_data)
hi_shared_variants = identify_shared_variants(hi_csv_data)

# Save the shared variants to new CSV files
ndm_shared_csv_path = 'NDM_shared_variants_5.csv'
hi_shared_csv_path = 'HI_shared_variants_5.csv'

ndm_shared_variants.to_csv(ndm_shared_csv_path, index=False)
hi_shared_variants.to_csv(hi_shared_csv_path, index=False)

print(f"NDM shared variants saved to {ndm_shared_csv_path}")
print(f"HI shared variants saved to {hi_shared_csv_path}")

NDM shared variants saved to NDM_shared_variants_5.csv
HI shared variants saved to HI_shared_variants_5.csv


In [13]:
import pandas as pd

# Load the DeNovo CSV files
ndm_csv_data = pd.read_csv('NDM_DeNovo5.csv')
hi_csv_data = pd.read_csv('HI_DeNovo5.csv')

# Function to identify shared variants with overlaps
def identify_shared_variants(df, overlap_threshold=0.3):
    # Sort by Chr, Region1, and Region2
    df = df.sort_values(by=['Chr', 'Region1', 'Region2'])

    # Create an empty DataFrame to store results
    shared_variants = pd.DataFrame()

    # Loop through each chromosome
    for chr in df['Chr'].unique():
        chr_df = df[df['Chr'] == chr]

        # Initialize a list to hold overlapping groups
        overlapping_groups = []

        # Initialize the first group with the first row
        current_group = [chr_df.iloc[0]]

        for i in range(1, len(chr_df)):
            prev_row = chr_df.iloc[i-1]
            curr_row = chr_df.iloc[i]

            # Calculate the overlap
            overlap_start = max(prev_row['Region1'], curr_row['Region1'])
            overlap_end = min(prev_row['Region2'], curr_row['Region2'])
            overlap_length = max(0, overlap_end - overlap_start + 1)

            # Calculate the lengths of the CNVs
            prev_length = prev_row['Region2'] - prev_row['Region1'] + 1
            curr_length = curr_row['Region2'] - curr_row['Region1'] + 1

            # Calculate the overlap percentage
            overlap_percentage = overlap_length / min(prev_length, curr_length)

            if overlap_percentage >= overlap_threshold:
                # If they overlap enough, add the current row to the group
                current_group.append(curr_row)
            else:
                # If they don't overlap enough, finalize the current group and start a new one
                overlapping_groups.append(pd.DataFrame(current_group))
                current_group = [curr_row]

        # Add the last group
        overlapping_groups.append(pd.DataFrame(current_group))

        # For each group of overlapping variants, count unique Sample_IDs
        for group in overlapping_groups:
            if group['Sample_ID'].nunique() > 1:
                shared_variants = pd.concat([shared_variants, group])

    return shared_variants

# Apply the function to both datasets with a 30% overlap threshold
ndm_shared_variants = identify_shared_variants(ndm_csv_data, overlap_threshold=0.3)
hi_shared_variants = identify_shared_variants(hi_csv_data, overlap_threshold=0.3)

# Save the shared variants to new CSV files
ndm_shared_csv_path = 'NDM_shared_variants_5_3.csv'
hi_shared_csv_path = 'HI_shared_variants_5_3.csv'

ndm_shared_variants.to_csv(ndm_shared_csv_path, index=False)
hi_shared_variants.to_csv(hi_shared_csv_path, index=False)

print(f"NDM shared variants saved to {ndm_shared_csv_path}")
print(f"HI shared variants saved to {hi_shared_csv_path}")

NDM shared variants saved to NDM_shared_variants_5_3.csv
HI shared variants saved to HI_shared_variants_5_3.csv


# Filtering out Inversions and Reformat Type column

In [10]:
import pandas as pd

# Paths to the CSV files
ndm_csv_file_path = 'NDM_shared_variants_5.csv'
hi_csv_file_path = 'HI_shared_variants_5.csv'

# Load the CSV files into DataFrames
ndm_data = pd.read_csv(ndm_csv_file_path)
hi_data = pd.read_csv(hi_csv_file_path)

# Function to filter out inversions and convert CNV types
def filter_and_convert_cnv(df):
    # Filter out rows where 'Type' column contains 'Inversion'
    df_filtered = df[~df['Type'].str.contains('Inversion', case=False, na=False)]
    
    # Replace 'Deletion' with 'DEL' and 'Duplication' with 'DUP' in the 'Type' column
    df_filtered['Type'] = df_filtered['Type'].replace({'Deletion': 'DEL', 'Duplication': 'DUP'})
    
    return df_filtered

# Apply the function to both DataFrames
filtered_ndm_data = filter_and_convert_cnv(ndm_data)
filtered_hi_data = filter_and_convert_cnv(hi_data)

# Save the filtered and converted data to new CSV files
filtered_ndm_csv_path = 'NDM_DUPDEL.csv'
filtered_hi_csv_path = 'HI_DUPDEL.csv'

filtered_ndm_data.to_csv(filtered_ndm_csv_path, index=False)
filtered_hi_data.to_csv(filtered_hi_csv_path, index=False)

print(f"NDM filtered data saved to {filtered_ndm_csv_path}")
print(f"HI filtered data saved to {filtered_hi_csv_path}")

NDM filtered data saved to NDM_DUPDEL.csv
HI filtered data saved to HI_DUPDEL.csv


/var/folders/c9/5211zt715d7151h7fkq093h40000gn/T/ipykernel_56566/991424124.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Type'] = df_filtered['Type'].replace({'Deletion': 'DEL', 'Duplication': 'DUP'})


In [12]:
import pandas as pd

# Load your CSV file
csv_file_path = 'HI_DUPDEL.csv'
data = pd.read_csv(csv_file_path)

# Select the required columns for BED format
bed_data = data[['Chr', 'Region1', 'Region2', 'Type','Sample_ID','Read_Count','Read_Width']]

# Save as a BED file
bed_file_path = 'HI_DUPDEL.bed'
bed_data.to_csv(bed_file_path, sep='\t', header=False, index=False)